In [1]:
import sys
import os
import math
from tqdm import tqdm
sys.path.append(os.path.join(os.path.expanduser('~'), 'projects'))
import j_utils.munging as mg
import seaborn as sns

ppath = os.path.join(os.path.expanduser('~'), 'projects', 'lendingclub', )
dpath = os.path.join(ppath,'data')
pd.options.display.max_columns = 999

In [2]:
check_cols = ['maturity_time', 'maturity_paid', 'maturity_time_stat_adj', 'maturity_paid_stat_adj',
              'target_loose', 'target_strict', 'loan_status', 'issue_d', 'end_d', 'id']
good_statuses = ['paid', 'current']

In [3]:
eval_loan_info = pd.read_feather(os.path.join(dpath,'eval_loan_info.fth'))
scaled_pmt_hist = pd.read_feather(os.path.join(dpath,'scaled_pmt_hist.fth'))
base_loan_info = pd.read_feather(os.path.join(dpath,'base_loan_info.fth'))
str_loan_info = pd.read_feather(os.path.join(dpath,'str_loan_info.fth'))

In [4]:
# add in issue_d
base_loan_info['issue_d'] = eval_loan_info['issue_d']

In [15]:
# get loans that are >= .95 maturity time, maturity paid, or status is in defaulted, charged_off, paid
trainable_loan_ids = eval_loan_info[(eval_loan_info['maturity_time_stat_adj'] >= .95) |
                               (eval_loan_info['maturity_paid_stat_adj'] >= .95) |
                               (eval_loan_info['loan_status'].isin(['paid', 'charged_off', 'defaulted']))
                              ]['id'].values

# fix step to data

In [6]:
# quick fix to eval_loan_info, future times see if this cell needs to be run
eval_loan_info['maturity_time_stat_adj'] = np.where(
    (eval_loan_info['maturity_time_stat_adj'] == -1) & (eval_loan_info['loan_status'] == 'paid'), 1,
         np.where(
             (eval_loan_info['maturity_time_stat_adj'] == -1) & (eval_loan_info['loan_status'] == 'current'), 
 eval_loan_info['maturity_time'], eval_loan_info['maturity_time_stat_adj']))

eval_loan_info['maturity_paid_stat_adj'] = np.where(
    (eval_loan_info['maturity_paid_stat_adj'] == -1) & (eval_loan_info['loan_status'] == 'paid'), 1,
         np.where(
             (eval_loan_info['maturity_paid_stat_adj'] == -1) & (eval_loan_info['loan_status'] == 'current'), 
 eval_loan_info['maturity_paid'], eval_loan_info['maturity_paid_stat_adj']))

# continue

In [16]:
trainable_loans = eval_loan_info[eval_loan_info['id'].isin(trainable_loan_ids)]
untrainable_loans = eval_loan_info[~eval_loan_info['id'].isin(trainable_loan_ids)]

In [28]:
# make sure untrainable loans actually look untrainable
untrainable_loans.groupby('loan_status',).apply(lambda x: x.sample(min(len(x), 3)))

end_d    issue_d  maturity_paid  maturity_time  \
loan_status                                                               
current     2018643 2019-06-01 2018-07-01       0.277522       0.305556   
            2307291 2019-06-01 2019-02-01       0.086818       0.111111   
            2275008 2019-06-01 2019-01-01       0.110663       0.138889   
grace_15    1206895 2019-06-01 2016-09-01       0.751400       0.916667   
            1398370 2019-06-01 2017-03-01       0.692811       0.750000   
            2301886 2019-06-01 2019-02-01       0.054802       0.111111   
late_30     2124361 2019-06-01 2018-09-01       0.220259       0.250000   
            1906759 2019-06-01 2018-04-01       0.307514       0.388889   
            1289494 2019-06-01 2016-12-01       0.466308       0.500000   

                     maturity_time_stat_adj  maturity_paid_stat_adj  \
loan_status                                                           
current     2018643                0.305556                0.277522   
            2307291                0.111111                0.086818   
            2275008                0.138889                0.110663   
grace_15    1206895                0.945833                0.838410   
            1398370                0.837500                0.800327   
            2301886                0.422222                0.385621   
late_30     2124361                0.730000                0.719293   
            1906759                0.780000                0.750705   
            1289494                0.820000                0.807871   

                     rem_to_be_paid  roi_simple  target_loose  target_strict  \
loan_status                                                                    
current     2018643    20132.439453    0.309336             0              0   
            2307291    32527.478516    0.123698             0              0   
            2275008     7303.747559    0.129833             0              0   
grace_15    1206895     7887.348633    0.953591             0              1   
            1398370    12058.179688    0.877263             0              1   
            2301886    12452.697266    0.072200             0              1   
late_30     2124361    11258.762695    0.265028             0              1   
            1906759     5713.670898    0.390352             0              1   
            1289494     9337.581055    0.679885             0              1   

                    loan_status         id      0.05      0.06      0.07  \
loan_status                                                                
current     2018643     current  136413546 -0.697245 -0.698614 -0.699974   
            2307291     current  147013618 -0.875955 -0.876154 -0.876354   
            2275008     current  145937068 -0.870919 -0.871186 -0.871452   
grace_15    1206895    grace_15   89896666 -0.068333 -0.079469 -0.090418   
            1398370    grace_15  100449537 -0.168723 -0.177669 -0.186482   
            2301886    grace_15  146825143 -0.891031 -0.891212 -0.891392   
late_30     2124361     late_30  140594225 -0.738458 -0.739440 -0.740416   
            1906759     late_30  131832476 -0.618275 -0.620137 -0.621987   
            1289494     late_30   94061429 -0.358697 -0.366208 -0.373596   

                         0.08      0.09       0.1      0.11      0.12  \
loan_status                                                             
current     2018643 -0.701325 -0.702666 -0.704000 -0.705324 -0.706640   
            2307291 -0.876552 -0.876750 -0.876948 -0.877145 -0.877341   
            2275008 -0.871717 -0.871981 -0.872244 -0.872506 -0.872768   
grace_15    1206895 -0.101184 -0.111770 -0.122181 -0.132420 -0.142489   
            1398370 -0.195163 -0.203715 -0.212140 -0.220441 -0.228619   
            2301886 -0.891571 -0.891750 -0.891928 -0.892106 -0.892283   
late_30     2124361 -0.741388 -0.742353 -0.743314 -0.744269 -0.745219   
            1906759 -0.623823 -0.625646 -0

In [18]:
ex = eval_loan_info[~eval_loan_info['loan_status'].isin(['paid', 'current', 'grace_15'])]

In [21]:
ex = ex[ex['target_strict'] != 1]
ex = ex.iloc[np.r_[0:4, -4:0]]

In [22]:
ex

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35
267396,2019-06-01,2014-03-01,0.997880,1.000000,0.970909,0.999103,57.751637,1.635008,0,0.0,late_30,12458945,0.444150,0.409697,0.376375,0.344141,0.312955,0.282776,0.253568,0.225294,0.197920,0.171412,0.145739,0.120869,0.096773,0.073424,0.050794,0.028856,0.007587,-0.013038,-0.033043,-0.052448,-0.071276,-0.089547,-0.107281,-0.124495,-0.141209,-0.157441,-0.173206,-0.188520,-0.203400,-0.217860,-0.231915
287554,2019-06-01,2014-04-01,0.983371,1.000000,1.000000,1.000000,353.217316,1.392518,1,0.0,charged_off,13477309,0.232296,0.203329,0.175299,0.148172,0.121913,0.096492,0.071876,0.048037,0.024946,0.002576,-0.019100,-0.040107,-0.060468,-0.080207,-0.099346,-0.117907,-0.135911,-0.153376,-0.170322,-0.186767,-0.202729,-0.218225,-0.233270,-0.247881,-0.262072,-0.275858,-0.289253,-0.302270,-0.314922,-0.327221,-0.339179
291285,2019-06-01,2014-05-01,0.982974,1.000000,0.970000,0.992783,345.350159,1.579284,0,0.0,late_30,13998262,0.397573,0.364721,0.332931,0.302166,0.272386,0.243554,0.215637,0.188601,0.162413,0.137042,0.112459,0.088635,0.065543,0.043157,0.021450,0.000400,-0.020018,-0.039826,-0.059044,-0.077695,-0.095798,-0.113372,-0.130435,-0.147006,-0.163100,-0.178736,-0.193927,-0.208690,-0.223038,-0.236987,-0.250549
296903,2019-06-01,2014-04-01,0.983274,1.000000,0.984444,0.998028,460.823639,1.545817,0,0.0,late_120,14579287,0.367956,0.335800,0.304685,0.274571,0.245422,0.217202,0.189876,0.163413,0.137780,0.112947,0.088884,0.065565,0.042963,0.021051,-0.000196,-0.020800,-0.040785,-0.060173,-0.078985,-0.097240,-0.114959,-0.132161,-0.148863,-0.165082,-0.180836,-0.196139,-0.211009,-0.225459,-0.239503,-0.253156,-0.266431
2374729,2019-06-01,2019-03-01,0.000000,0.083333,0.736667,0.000000,24081.931641,0.000000,0,0.0,late_120,149405288,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2374746,2019-06-01,2019-03-01,0.000000,0.083333,0.736667,0.000000,22115.998047,0.000000,0,0.0,late_120,149406489,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2374971,2019-06-01,2019-03-01,0.000000,0.083333,1.000000,1.000000,4810.364746,0.000000,1,0.0,charged_off,149421188,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2375679,2019-06-01,2019-03-01,0.000000,0.083333,0.736667,0.000000,6689.418457,0.000000,0,0.0,late_120,149469048,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000


In [ ]:
'''
things that I always expect to be true about eval_loan_info
1) if loan_status is not paid,current,grace_15, then target_strict must be 1
2) if loan_status isn't current, stat_adj values should be higher than non_stat adj values
3) if loan_status is a bad status, maturity_time_stat_adj and maturity_paid_stat_adj should be higher than
unadjusted version
3) 
'''